In [62]:
import requests
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas

In [63]:
print("On what topic do you want questions?")
topic = input()
print("How many questions do you want?")
num_questions = int(input())

On what topic do you want questions?
How many questions do you want?


ValueError: invalid literal for int() with base 10: ''

In [64]:
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

In [65]:
wikidata_property_example = "P1855"

In [66]:
def display_as_table(results):
    df = pandas.DataFrame.from_dict(results["results"]["bindings"])
    df = df.applymap(lambda x: x["value"])
    print(df)

In [67]:
def find_uri_by_label(label):
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
    
    # SPARQL query to find the URI for a given label
    query = """
    SELECT ?item WHERE {
      ?item rdfs:label "%s"@en.
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
    }
    LIMIT 1
    """ % label.replace('"', '\"')  # Basic string sanitization
    
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    
    try:
        results = sparql.query().convert()
        for result in results["results"]["bindings"]:
            return result["item"]["value"]  # Return the first matching URI
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [68]:
def run_sparql(sparql_query):
    print(sparql_query)
    # Set the query and the return format (JSON)
    sparql.setQuery(sparql_query)
    sparql.setReturnFormat(JSON)

    # Perform the query and convert the result to a Python dictionary
    results = sparql.query().convert()
    return results

In [69]:
def find_Q_by_uri(uri):
    return uri.split("/")[-1]

In [70]:
def find_Q_by_label(label):
    return find_Q_by_uri(find_uri_by_label(label))

In [85]:
# I checked all of the ?predicates offered by ChatGPT and they are sensible
sensible_sister_predicates = "wdt:P279 wdt:P31 wdt:P361 wdt:P101 wdt:P361 wdt:P921 wdt:P131 wdt:P150 wdt:P21"
def sister_topic(topicQ, predicates=sensible_sister_predicates):
    
    # SPARQL query to find topics that are similar to the given topic
    query = """
    SELECT DISTINCT ?item WHERE {
        VALUES ?predicates { """ + predicates + """ }
        wd:""" + topicQ +  """ ?predicates ?class.
        ?item ?predicates ?class.
        FILTER(?item != wd:""" + topicQ +  """)
        
        SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
    }
    LIMIT 10
    """
    return run_sparql(query)

In [86]:
display_as_table(sister_topic(find_Q_by_label("God")))


    SELECT DISTINCT ?item WHERE {
        VALUES ?predicates { wdt:P279 wdt:P31 wdt:P361 wdt:P101 wdt:P361 wdt:P921 wdt:P131 wdt:P150 wdt:P21 }
        wd:Q190 ?predicates ?class.
        ?item ?predicates ?class.
        FILTER(?item != wd:Q190)
        
        SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
    }
    LIMIT 10
    
                                     item
0   http://www.wikidata.org/entity/Q71684
1   http://www.wikidata.org/entity/Q75799
2   http://www.wikidata.org/entity/Q82198
3   http://www.wikidata.org/entity/Q83090
4   http://www.wikidata.org/entity/Q86901
5  http://www.wikidata.org/entity/Q105173
6  http://www.wikidata.org/entity/Q106782
7  http://www.wikidata.org/entity/Q118289
8  http://www.wikidata.org/entity/Q127570
9  http://www.wikidata.org/entity/Q128593


/var/folders/g_/12fwrqfs2lgcf9rnql4m440dx849vl/T/ipykernel_27706/1413346344.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x["value"])


In [87]:
def sparql_question(topicQ, predicateQ):
    # Define the SPARQL query
    sparql_query = """
    SELECT ?thing WHERE {
      
    ?thing""" + " wdt:" + predicateQ + " wd:" + topicQ + """.

    SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
    }
    LIMIT 10
    """
    print("running query: " + sparql_query)

    return run_sparql(sparql_query)

In [21]:
find_Q_by_label(topic)

'Q3624078'

In [32]:
sparql_question(find_Q_by_label(topic), find_Q_by_label("instance of"))

running query: 
    SELECT ?thing WHERE {
      
    ?thing wdt:P31 wd:Q3624078.

    SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
    }
    LIMIT 10
    


{'head': {'vars': ['thing']},
 'results': {'bindings': [{'thing': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q16'}},
   {'thing': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q17'}},
   {'thing': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q20'}},
   {'thing': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q27'}},
   {'thing': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q28'}},
   {'thing': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q29'}},
   {'thing': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q30'}},
   {'thing': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q31'}},
   {'thing': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q32'}},
   {'thing': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q33'}}]}}

In [ ]:

def get_wikidata_item(item_id):
    # Wikidata API endpoint
    url = "https://www.wikidata.org/w/api.php"
    
    # Parameters for the API request
    params = {
        "action": "wbgetentities",  # Action to get data about entities
        "ids": item_id,  # ID of the Wikidata item (e.g., Q64 for Berlin)
        "format": "json",  # Response format
        "props": "labels|descriptions|aliases",  # Properties to retrieve: labels, descriptions, and aliases
        "languages": "en"  # Language filter
    }
    
    # Make the GET request to the Wikidata API
    response = requests.get(url, params=params)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        
        # Access the item's data
        item_data = data['entities'][item_id]
        
        # Extract and print the label, description, and aliases
        label = item_data['labels']['en']['value']
        description = item_data['descriptions']['en']['value']
        aliases = [alias['value'] for alias in item_data['aliases']['en']]
        
        print(f"Label: {label}")
        print(f"Description: {description}")
        print(f"Aliases: {', '.join(aliases)}")
    else:
        print("Failed to retrieve data")

# Example usage
get_wikidata_item("Q64")

Label: Berlin
Description: federated state, capital and largest city of Germany
Aliases: Berlin, Germany, Berlin (Germany), DE-BE
